In [1]:
# Step 1: Generate a small synthetic fraud detection dataset for demonstration
import numpy as np
import pandas as pd

np.random.seed(42)

# Generate 1000 samples
n_samples = 1000
# Features: amount, transaction_time, is_foreign, is_high_risk_country
amount = np.random.exponential(scale=100, size=n_samples)
transaction_time = np.random.randint(0, 24, size=n_samples)  # hour of day
is_foreign = np.random.binomial(1, 0.1, size=n_samples)  # 10% foreign
is_high_risk_country = np.random.binomial(1, 0.05, size=n_samples)  # 5% high risk

# Generate labels: fraud is more likely for high amount, foreign, high risk country, odd hours
fraud = (
        (amount > 200).astype(int) +
        (is_foreign == 1).astype(int) +
        (is_high_risk_country == 1).astype(int) +
        ((transaction_time < 6) | (transaction_time > 22)).astype(int)
)
# If sum of risk factors >= 2, label as fraud
labels = (fraud >= 2).astype(int)

# Create DataFrame
fraud_df = pd.DataFrame({
    'amount': amount,
    'transaction_time': transaction_time,
    'is_foreign': is_foreign,
    'is_high_risk_country': is_high_risk_country,
    'fraud': labels
})

print('Sample of synthetic fraud detection data:')
display(fraud_df.head())
print('Fraud distribution:')
print(fraud_df["fraud"].value_counts())

# Save the generated synthetic fraud detection DataFrame to a CSV file for later use.
fraud_df.to_csv('synthetic_fraud_data.csv', index=False)
print('Data saved to synthetic_fraud_data.csv')


Sample of synthetic fraud detection data:


,amount,transaction_time,is_foreign,is_high_risk_country,fraud
0,46.926809,14,0,0,0
1,301.012143,11,1,0,1
2,131.674569,15,0,0,0
3,91.294255,23,1,0,1
4,16.962487,18,0,0,0


Fraud distribution:
fraud
0    897
1    103
Name: count, dtype: int64
Data saved to synthetic_fraud_data.csv


In [ ]:
# Step 2: Read the synthetic fraud detection data from file
import pandas as pd
fraud_df = pd.read_csv('synthetic_fraud_data.csv')
print('Loaded data sample:')
display(fraud_df.head())
print('Fraud distribution:')
print(fraud_df['fraud'].value_counts())


In [ ]:
# Step 3: Exploratory Data Analysis (EDA)
import matplotlib.pyplot as plt

# 1. Check for missing values
print('Missing values in each column:')
print(fraud_df.isnull().sum())

# 2. Summary statistics
print('\nSummary statistics:')
print(fraud_df.describe())

# 3. Visualize feature distributions
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0].hist(fraud_df['amount'], bins=30, color='skyblue')
axs[0, 0].set_title('Amount Distribution')
axs[0, 1].hist(fraud_df['transaction_time'], bins=24, color='orange')
axs[0, 1].set_title('Transaction Time Distribution')
axs[1, 0].bar(['No', 'Yes'], fraud_df['is_foreign'].value_counts().sort_index(), color='green')
axs[1, 0].set_title('Is Foreign')
axs[1, 1].bar(['No', 'Yes'], fraud_df['is_high_risk_country'].value_counts().sort_index(), color='red')
axs[1, 1].set_title('Is High Risk Country')
plt.tight_layout()
plt.show()

# 4. Check class balance
print('\nFraud class balance:')
print(fraud_df['fraud'].value_counts(normalize=True))


In [ ]:
# Step 4: Data Preprocessing (train-test split and scaling)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Split features and target
y = fraud_df['fraud']
X = fraud_df.drop('fraud', axis=1)

# 2. Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Scale numerical features
scaler = StandardScaler()

X_train[['amount', 'transaction_time']] = scaler.fit_transform(X_train[['amount', 'transaction_time']])
X_test[['amount', 'transaction_time']] = scaler.transform(X_test[['amount', 'transaction_time']])

# 4. Show shapes
print(f'Training set shape: {X_train.shape}, {y_train.shape}')
print(f'Test set shape: {X_test.shape}, {y_test.shape}')
print('Sample of scaled training data:')
display(X_train.head())

# Save train and test data for later use
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
print('Train and test data saved as X_train.csv, X_test.csv, y_train.csv, y_test.csv')


In [10]:
# Step 5: Train and evaluate a Logistic Regression model for fraud detection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd

# Load train and test data (in case running this cell independently)
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()

# Train logistic regression model
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

# Predict on test set
y_pred = logreg.predict(X_test)
y_proba = logreg.predict_proba(X_test)[:, 1]

# Evaluation
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))
print('\nClassification Report:')
print(classification_report(y_test, y_pred, digits=3))
print(f'ROC AUC Score: {roc_auc_score(y_test, y_proba):.3f}')


Confusion Matrix:
[[178   1]
 [ 12   9]]

Classification Report:
              precision    recall  f1-score   support

           0      0.937     0.994     0.965       179
           1      0.900     0.429     0.581        21

    accuracy                          0.935       200
   macro avg      0.918     0.711     0.773       200
weighted avg      0.933     0.935     0.924       200

ROC AUC Score: 0.930


In [9]:
# Step 6: Train and evaluate a Neural Network for fraud detection
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd

# Load train and test data (in case running this cell independently)
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv').values.ravel()
y_test = pd.read_csv('y_test.csv').values.ravel()

# Build a simple neural network model
nn_model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=1, validation_split=0.1)

# Predict on test set
y_pred_nn = (nn_model.predict(X_test) > 0.5).astype(int).ravel()
y_proba_nn = nn_model.predict(X_test).ravel()

# Evaluation
print('Confusion Matrix (Neural Network):')
print(confusion_matrix(y_test, y_pred_nn))
print('\nClassification Report (Neural Network):')
print(classification_report(y_test, y_pred_nn, digits=3))
print(f'ROC AUC Score (Neural Network): {roc_auc_score(y_test, y_proba_nn):.3f}')


/Users/sefa.balik/projects/datascience/neuralnet/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Epoch 1/30


/Users/sefa.balik/projects/datascience/neuralnet/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6929 - loss: 0.6818 - val_accuracy: 0.8750 - val_loss: 0.6300
Epoch 2/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8943 - loss: 0.5955 - val_accuracy: 0.8750 - val_loss: 0.5656
Epoch 3/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8917 - loss: 0.5112 - val_accuracy: 0.8750 - val_loss: 0.5119
Epoch 4/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8913 - loss: 0.4600 - val_accuracy: 0.8750 - val_loss: 0.4627
Epoch 5/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8982 - loss: 0.3976 - val_accuracy: 0.8750 - val_loss: 0.4212
Epoch 6/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8970 - loss: 0.3656 - val_accuracy: 0.8750 - val_loss: 0.3871
Epoch 7/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8907 - loss: 0.3432 - val_accuracy: 0.8750 - val_loss: 0.3610
Epoch 8/30
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8873 - loss: 0.3243 - val_accuracy: 0.8750 - val_loss: 0.3422
Epo